In [ ]:
from torchvision import datasets
import matplotlib.pyplot as plt
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torch

In [ ]:
datapath = 'C:/Users/MSI/Desktop/Data Science/pytorch'
cifar10 = datasets.CIFAR10(datapath, train = True, download = True)
cifar10_val = datasets.CIFAR10(datapath, train = False, download=True)

In [ ]:
conv = nn.Conv2d(3, 16, kernel_size=3)
conv

In [ ]:
#check the weights
conv.weight.shape

In [ ]:
conv.bias.shape #it’s a constant value we add to each channel of the output image

In [ ]:
#GET THE CIFAR DATASE
tensor_cifar10 = datasets.CIFAR10(datapath, download = True, train = True, transform = transforms.ToTensor())
tensor_cifar10_val = datasets.CIFAR10(datapath, download = True, train = False, transform = transforms.ToTensor())

In [ ]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']

cifar2 = [(img, label_map[label]) 
          for img, label in tensor_cifar10
          if label in [0, 2]]

cifar2_val = [(img, label_map[label])
              for img, label in tensor_cifar10_val
              if label in [0, 2]]

In [ ]:
#Make a little run of the CONV just to see how it operates

img, _ = cifar2[0]
output = conv(img.unsqueeze(0))
print(img.unsqueeze(0).shape)
print(output.shape)

In [ ]:
plt.imshow(output[0,0].detach()) #The fact that our output image is smaller than the input is a side effect of deciding what
#                                 to do at the boundary of the image. (the i00 dimentions)

In [ ]:
#let's fix that

conv_pad = nn.Conv2d(3,1, kernel_size=3, padding=1)
output= conv_pad(img.unsqueeze(0))
img.unsqueeze(0).shape

In [ ]:
#Let's create a model

model = nn.Sequential(
                nn.Conv2d(3, 16, kernel_size=3, padding=1),
                nn.Tanh(),
                nn.MaxPool2d(2),
                nn.Conv2d(16, 8, kernel_size=3, padding=1),
                nn.Tanh(),
                nn.MaxPool2d(2))

#But! the problem with the Sequential way is that it doesn't have a ".reshape" option for the 1D output, so the model won't run
#we need to use the 

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 2)
    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8)
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out        

In [ ]:
import torch.nn.functional as F

In [ ]:
##The functional API

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding =1)
        self.fc1 = nn.Linear(8*8*8, 32)
        self.fc2 = nn.Linear(32, 2)
    
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8*8*8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
model = Net()

In [ ]:
model(img.unsqueeze(0))

## Training our convnet

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=False)

val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)

In [ ]:
def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        with torch.no_grad(): #We do not want gradients  here, as we will not want to  update the parameters.
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())
            print("Accuracy {}: {:.2f}".format(name , correct / total))

In [ ]:
validate(model, train_loader, val_loader)

In [ ]:
### save the model

torch.save(model.state_dict(), 'D:/Data Science projects/pytorch'+ 'chapter8.pt')

In [ ]:
loaded_model = Net()
loaded_model.load_state_dict(torch.load(data_path+ 'birds_vs_airplanes.pt'))

In [ ]:
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print(f"Training on device {device}.")